In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import folium
import geopy

In [31]:
data=pd.read_csv('C:/Users/aditya/Desktop/zomato.csv')
data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [32]:
data_1=data[['location','rate','votes','approx_cost(for two people)']]
data_1.head()

,location,rate,votes,approx_cost(for two people)
0,Banashankari,4.1/5,775,800
1,Banashankari,4.1/5,787,800
2,Banashankari,3.8/5,918,800
3,Banashankari,3.7/5,88,300
4,Basavanagudi,3.8/5,166,600


In [33]:
data_1.drop_duplicates(inplace=True)

C:\Users\aditya\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
data=data_1
#REMOVING NAN VALUES
data.isnull().sum()
data.dropna(how='any',inplace=True)
data.head()

,location,rate,votes,approx_cost(for two people)
0,Banashankari,4.1/5,775,800
1,Banashankari,4.1/5,787,800
2,Banashankari,3.8/5,918,800
3,Banashankari,3.7/5,88,300
4,Basavanagudi,3.8/5,166,600


In [35]:
data.rename(columns={'approx_cost(for two people)':'cost'},inplace=True)
data['cost']=data['cost'].astype(str)
data['cost']=data['cost'].apply(lambda x:x.replace(',',''))
data['cost']=data['cost'].astype(float)

In [36]:
data['rate']=data['rate'].replace('NEW',np.nan)
data['rate'].dropna(inplace=True)

In [37]:
data['rate'].replace('-','2.55/5',inplace=True)
remove_slash=lambda x:x.replace('/5','') if type(x)==np.str else x
data.rate=data.rate.apply(remove_slash).str.strip().astype('float')
data['rate']

0        4.1
1        4.1
2        3.8
3        3.7
4        3.8
        ... 
51624    3.3
51628    4.2
51631    3.4
51632    3.3
51677    2.5
Name: rate, Length: 19065, dtype: float64

In [38]:
data.head()

,location,rate,votes,cost
0,Banashankari,4.1,775,800.0
1,Banashankari,4.1,787,800.0
2,Banashankari,3.8,918,800.0
3,Banashankari,3.7,88,300.0
4,Basavanagudi,3.8,166,600.0


# PLACES WITH BEST RATING (AVERAGE)

In [48]:
data_1=data[['location','rate']]

In [49]:
data_1.head(10)

,location,rate
0,Banashankari,4.1
1,Banashankari,4.1
2,Banashankari,3.8
3,Banashankari,3.7
4,Basavanagudi,3.8
5,Basavanagudi,3.8
6,Mysore Road,3.6
7,Banashankari,4.6
8,Banashankari,4.0
9,Banashankari,4.2


In [52]:
data_2=data_1.groupby('location').rate.mean()

In [55]:
data_3=pd.DataFrame(data_2)

In [56]:
data_3.head()

,rate
location,
BTM,3.637437
Banashankari,3.669204
Banaswadi,3.496250
Bannerghatta Road,3.549810
Basavanagudi,3.660265


In [57]:
data_3=data_3.reset_index()

In [58]:
data_3.head()

,location,rate
0,BTM,3.637437
1,Banashankari,3.669204
2,Banaswadi,3.496250
3,Bannerghatta Road,3.549810
4,Basavanagudi,3.660265


In [61]:
data_3.shape

(92, 2)

In [62]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = 'app')

In [63]:
lat_lon = []
for location in data_3['location']:
    location = geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:
        geo = (location.latitude, location.longitude)
        lat_lon.append(geo)

In [64]:
data_3['geo_loc'] = lat_lon

In [78]:
data_3.head()

,location,rate,geo_loc
0,BTM,3.637437,"(45.95485055, -112.49659530324134)"
1,Banashankari,3.669204,"(15.8876779, 75.7046777)"
2,Banaswadi,3.496250,"(13.0141618, 77.6518539)"
3,Bannerghatta Road,3.549810,"(12.887979, 77.5970812)"
4,Basavanagudi,3.660265,"(12.9417261, 77.5755021)"


In [84]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   location  92 non-null     object 
 1   rate      92 non-null     float64
 2   geo_loc   91 non-null     object 
dtypes: float64(1), object(2)
memory usage: 2.3+ KB


In [85]:
data_3.dropna(inplace=True)

In [86]:
data_3.head()

,location,rate,geo_loc
0,BTM,3.637437,"(45.95485055, -112.49659530324134)"
1,Banashankari,3.669204,"(15.8876779, 75.7046777)"
2,Banaswadi,3.496250,"(13.0141618, 77.6518539)"
3,Bannerghatta Road,3.549810,"(12.887979, 77.5970812)"
4,Basavanagudi,3.660265,"(12.9417261, 77.5755021)"


In [92]:
lat=[]
for i in data_3['geo_loc']:
    lat.append(i[0])
    print(lat)

[45.95485055]
[45.95485055, 15.8876779]
[45.95485055, 15.8876779, 13.0141618]
[45.95485055, 15.8876779, 13.0141618, 12.887979]
[45.95485055, 15.8876779, 13.0141618, 12.887979, 12.9417261]
[45.95485055, 15.8876779, 13.0141618, 12.887979, 12.9417261, 12.9932739]
[45.95485055, 15.8876779, 13.0141618, 12.887979, 12.9417261, 12.9932739, 12.93577245]
[45.95485055, 15.8876779, 13.0141618, 12.887979, 12.9417261, 12.9932739, 12.93577245, 12.9089453]
[45.95485055, 15.8876779, 13.0141618, 12.887979, 12.9417261, 12.9932739, 12.93577245, 12.9089453, 40.28745]
[45.95485055, 15.8876779, 13.0141618, 12.887979, 12.9417261, 12.9932739, 12.93577245, 12.9089453, 40.28745, 33.5935063]
[45.95485055, 15.8876779, 13.0141618, 12.887979, 12.9417261, 12.9932739, 12.93577245, 12.9089453, 40.28745, 33.5935063, 17.2510682]
[45.95485055, 15.8876779, 13.0141618, 12.887979, 12.9417261, 12.9932739, 12.93577245, 12.9089453, 40.28745, 33.5935063, 17.2510682, 13.0101286]
[45.95485055, 15.8876779, 13.0141618, 12.887979, 12

In [93]:
len(lat)

91

In [96]:
lat

[45.95485055,
 15.8876779,
 13.0141618,
 12.887979,
 12.9417261,
 12.9932739,
 12.93577245,
 12.9089453,
 40.28745,
 33.5935063,
 17.2510682,
 13.0101286,
 51.373656,
 39.76880625,
 51.5227809,
 31.89376,
 12.9624669,
 13.0215466,
 12.945245,
 -6.2659285,
 12.996845,
 13.0358698,
 18.1475,
 13.0382184,
 13.0258087,
 12.773175,
 12.967576,
 12.9732913,
 34.977289,
 12.2655944,
 11.9917786,
 27.64392675,
 12.9678074,
 13.007516,
 12.9846713,
 13.0221416,
 13.0093455,
 12.5607431,
 12.9176571,
 13.2923988,
 14.5395813,
 12.9243509,
 12.9271867,
 12.93433385,
 12.9343774,
 12.9400321,
 12.9302645,
 12.9417812,
 12.9081487,
 12.957998,
 40.7652844,
 17.9895618,
 12.945048,
 1.2847055,
 13.0027353,
 12.9552572,
 12.3872141,
 12.9546741,
 13.2227,
 13.0346384,
 13.0217151,
 1.3061622,
 13.194416,
 13.0329419,
 13.0227204,
 1.306731,
 12.9882338,
 12.9274413,
 38.7385916,
 50.7721586,
 15.8782951,
 13.0621474,
 23.1485712,
 38.7801076,
 12.9242381,
 12.9931876,
 12.9575547,
 18.5322493,
 13.06

In [98]:
data_3['lat']=lat

In [99]:
long=[]
for i in data_3['geo_loc']:
    long.append(i[1])
    print(long)

[-112.49659530324134]
[-112.49659530324134, 75.7046777]
[-112.49659530324134, 75.7046777, 77.6518539]
[-112.49659530324134, 75.7046777, 77.6518539, 77.5970812]
[-112.49659530324134, 75.7046777, 77.6518539, 77.5970812, 77.5755021]
[-112.49659530324134, 75.7046777, 77.6518539, 77.5970812, 77.5755021, 77.5388099]
[-112.49659530324134, 75.7046777, 77.6518539, 77.5970812, 77.5755021, 77.5388099, 77.66676103753434]
[-112.49659530324134, 75.7046777, 77.6518539, 77.5970812, 77.5755021, 77.5388099, 77.66676103753434, 77.6239038]
[-112.49659530324134, 75.7046777, 77.6518539, 77.5970812, 77.5755021, 77.5388099, 77.66676103753434, 77.6239038, -76.964526]
[-112.49659530324134, 75.7046777, 77.6518539, 77.5970812, 77.5755021, 77.5388099, 77.66676103753434, 77.6239038, -76.964526, -79.0345627]
[-112.49659530324134, 75.7046777, 77.6518539, 77.5970812, 77.5755021, 77.5388099, 77.66676103753434, 77.6239038, -76.964526, -79.0345627, 80.1651978]
[-112.49659530324134, 75.7046777, 77.6518539, 77.5970812, 77.

In [101]:
data_3['long']=long

In [102]:
data_3.head()

,location,rate,geo_loc,lat,long
0,BTM,3.637437,"(45.95485055, -112.49659530324134)",45.954851,-112.496595
1,Banashankari,3.669204,"(15.8876779, 75.7046777)",15.887678,75.704678
2,Banaswadi,3.496250,"(13.0141618, 77.6518539)",13.014162,77.651854
3,Bannerghatta Road,3.549810,"(12.887979, 77.5970812)",12.887979,77.597081
4,Basavanagudi,3.660265,"(12.9417261, 77.5755021)",12.941726,77.575502


In [113]:
data_3.columns###########geo loc deleted

Index(['location', 'rate', 'lat', 'long'], dtype='object')

In [114]:
data_3.head()

,location,rate,lat,long
0,BTM,3.637437,45.954851,-112.496595
1,Banashankari,3.669204,15.887678,75.704678
2,Banaswadi,3.496250,13.014162,77.651854
3,Bannerghatta Road,3.549810,12.887979,77.597081
4,Basavanagudi,3.660265,12.941726,77.575502


In [115]:
def generateBaseMap(default_location=[12.9716,77.5946], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [124]:
base_map=generateBaseMap()
base_map


In [125]:
from folium.plugins import HeatMap

In [127]:
HeatMap(data_3[['lat','long','rate']].values.tolist(),zoom = 20, radius = 15).add_to(base_map)

In [129]:
base_map

In [142]:
from folium import plugins
markers = plugins.MarkerCluster(locations=[[lat,long]],popups=locations)
base_map.add_child(markers)

ValueError: Expected two (lat, lon) values for location, instead got: [45.95485055, 15.8876779, 13.0141618, 12.887979, 12.9417261, 12.9932739, 12.93577245, 12.9089453, 40.28745, 33.5935063, 17.2510682, 13.0101286, 51.373656, 39.76880625, 51.5227809, 31.89376, 12.9624669, 13.0215466, 12.945245, -6.2659285, 12.996845, 13.0358698, 18.1475, 13.0382184, 13.0258087, 12.773175, 12.967576, 12.9732913, 34.977289, 12.2655944, 11.9917786, 27.64392675, 12.9678074, 13.007516, 12.9846713, 13.0221416, 13.0093455, 12.5607431, 12.9176571, 13.2923988, 14.5395813, 12.9243509, 12.9271867, 12.93433385, 12.9343774, 12.9400321, 12.9302645, 12.9417812, 12.9081487, 12.957998, 40.7652844, 17.9895618, 12.945048, 1.2847055, 13.0027353, 12.9552572, 12.3872141, 12.9546741, 13.2227, 13.0346384, 13.0217151, 1.3061622, 13.194416, 13.0329419, 13.0227204, 1.306731, 12.9882338, 12.9274413, 38.7385916, 50.7721586, 15.8782951, 13.0621474, 23.1485712, 38.7801076, 12.9242381, 12.9931876, 12.9575547, 18.5322493, 13.0646907, 5

In [141]:
locations=list(data['location'])